# 1. Acquiring texts

## Exercise 1.1

1. Download all the text files that are listed in the following dictionary. 

`
    gutenberg_files = {
    'https://www.gutenberg.org/files/98/98-0.txt' :
        'A Tale of Two Cities',
    'https://www.gutenberg.org/files/580/580-0.txt':
        'The Pickwick Papers'
   }
`

Save these files in a folder named 'Texts'. In Python, you can make new folders using the `os` package, as follows:

`
os.mkdir('Texts')
`








In [ ]:
import os
import requests
import re
from tdmh import *

gutenberg_files = {
    'https://www.gutenberg.org/files/98/98-0.txt' :
        'A Tale of Two Cities',
    'https://www.gutenberg.org/files/580/580-0.txt':
        'The Pickwick Papers'
   }

dir = 'Texts'
if not os.path.exists(dir): 
    os.mkdir(dir)
    
#os.mkdir(dir)


for url in gutenberg_files:
    print("Downloading " + gutenberg_files[url] + " ...")
    response = requests.get(url)
    title = re.sub( r'\s+' , '_' ,  gutenberg_files[url] )

    if response:
        response.encoding = 'utf-8'
        full_text = remove_pg_boilerplate(response.text)      
        
        path = os.path.join( dir , f'{title}.txt' )
        out = open( path , 'w' , encoding = 'utf-8')

        out.write( full_text.strip() )
        out.close()

print('\nDone!')  



## Exercise 1.2.

Potentially, you can acquire texts using Web Scraping. 

The webpage below offers access to the complete work of H.P. Lovecraft. 
http://www.hplovecraft.com/writings/texts/

Can you write code in Python to download all the texts that are listed on this page?  

In [ ]:
from bs4 import BeautifulSoup
import requests
import re


dir = 'Lovecraft'
if not os.path.exists(dir): 
    os.mkdir(dir)

baseUrl = "http://www.hplovecraft.com/writings/texts/"

soup = ""

response = requests.get( baseUrl )

soup = BeautifulSoup( response.text ,"lxml")
links = soup.find_all("a")


for l in links:

    if l.get("href") is not None:
        
        ## The body of <a> contains the title 
        if l.string is not None:
            linktext = l.string
        else:
            linktext = l.get("href")
            linktext = linktext[ linktext.rindex('/') + 1 : ]
            linktext = re.sub( r'[.]aspx' , '' , linktext )
            
        textUrl = baseUrl + l.get("href")

        if re.search( 'fiction' , textUrl ):
            print(f"{linktext}: {textUrl}")
            response = requests.get( textUrl )
            soup = BeautifulSoup( response.text ,"lxml")
            body = soup.find("body")

            fullText = body.get_text()
            fullText = re.sub( 'By H. P. Lovecraft' , ' by H. P. Lovecraft ' , fullText )

            fullText = fullText[fullText.index('About This Site') + 15 : ]
            fullText = fullText[ 0 : fullText.rindex('Return to ') ]
            fullText = fullText.strip()
            
            path = os.path.join( dir , linktext + '.txt' )

            out = open( path , 'w' , encoding = 'utf-8' )
            out.write( fullText )
            out.close()
